In [ ]:
# !pip install -U langchain
!pip install langchain langchain-community langchain-core
!pip install langchain-openai openai
!pip install langchain-deepseek
!pip install deepseek-api python-dotenv

# Installing the OpenAI integration
#!pip install -U langchain-openai
# Installing the Anthropic integration
# !pip install -U langchain-anthropic

!pip show langchain-core

In [2]:
import os
from langchain.tools import tool, ToolRuntime
from langchain_openai import ChatOpenAI


# 设置 DeepSeek 的 API 密钥（LangChain-OpenAI 仍然会查找 OPENAI_API_KEY）
# 您可以尝试将 DEEPSEEK_API_KEY 的值赋给 OPENAI_API_KEY 环境变量
os.environ["OPENAI_API_KEY"] = os.environ.get("DEEPSEEK_API_KEY", "your_deepseek_api_key_here") # 确保这个值是 DeepSeek 的 key

# 关键：指定 DeepSeek 的 API 基础 URL
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"

model = ChatOpenAI(
    model="deepseek-chat", # 使用 DeepSeek 的模型名称
    openai_api_base=DEEPSEEK_BASE_URL, # 指定 DeepSeek 的 URL
    temperature=0.7
)

In [4]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=model,
    # tools=[weather_tool, calculator_tool],
    middleware=[
        SummarizationMiddleware(
            model=model,
            max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
            summary_prompt="Custom prompt for summarization...",  # Optional
        ),
    ],
)

In [10]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver


agent = create_agent(
    model=model,
    # tools=[read_email_tool, send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # Require approval, editing, or rejection for sending emails
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                # Auto-approve reading emails
                "read_email_tool": False,
            }
        ),
    ],
)

In [13]:
# !pip install langchain_anthropic
# from langchain_anthropic import ChatAnthropic
from langchain_anthropic.middleware import AnthropicPromptCachingMiddleware
from langchain.agents import create_agent
from langchain.messages import HumanMessage, AIMessage, SystemMessage


LONG_PROMPT = """
Please be a helpful assistant.

<Lots more context ...>
"""

agent = create_agent(
    model=model,
    system_prompt=LONG_PROMPT,
    middleware=[AnthropicPromptCachingMiddleware(ttl="5m")],
)

# cache store
agent.invoke({"messages": [HumanMessage("Hi, my name is Bob")]})

# cache hit, system prompt is cached
agent.invoke({"messages": [HumanMessage("What's my name?")]})

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='c6c4ff9a-20bc-46f6-9aaa-601f7859df7c'),
  AIMessage(content="I'm sorry, but I don't have access to your name or any personal information unless you've shared it with me in this conversation. If you'd like, you can tell me your name, and I can use it in our conversation!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 23, 'total_tokens': 72, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 23}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '9758f2b3-369c-41d3-a5af-a3e143d420a1', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3911581c-bef1-4993-8fdb-a3e7bb1618d0-0', usage_metadata={'input_tokens': 23, 'output_tokens': 49, '

In [15]:
from langchain.agents import create_agent
from langchain.agents.middleware import ModelCallLimitMiddleware


agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        ModelCallLimitMiddleware(
            thread_limit=10,  # Max 10 calls per thread (across runs)
            run_limit=5,  # Max 5 calls per run (single invocation)
            exit_behavior="end",  # Or "error" to raise exception
        ),
    ],
)

In [18]:
from langchain.agents import create_agent
from langchain.agents.middleware import ToolCallLimitMiddleware


# Limit all tool calls
global_limiter = ToolCallLimitMiddleware(thread_limit=20, run_limit=10)

# Limit specific tool
search_limiter = ToolCallLimitMiddleware(
    tool_name="search",
    thread_limit=5,
    run_limit=3,
)

agent = create_agent(
    model=model,
    tools=[],
    middleware=[global_limiter, search_limiter],
)

In [19]:
from langchain.agents import create_agent
from langchain.agents.middleware import ModelFallbackMiddleware


agent = create_agent(
    model=model,  # Primary model
    tools=[],
    middleware=[
        ModelFallbackMiddleware(
            model,  # Try first on error
        ),
    ],
)

In [20]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware


agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        # Redact emails in user input
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        # Mask credit cards (show last 4 digits)
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        # Custom PII type with regex
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",  # Raise error if detected
        ),
    ],
)

In [23]:
from langchain.agents import create_agent
from langchain.agents.middleware import TodoListMiddleware
from langchain.messages import HumanMessage


agent = create_agent(
    model=model,
    tools=[],
    middleware=[TodoListMiddleware()],
)

result = agent.invoke({"messages": [HumanMessage("Help me refactor my codebase")]})
# print(result["todos"])  # Array of todo items with status tracking
print(result["messages"])  # Array of todo items with status tracking

[HumanMessage(content='Help me refactor my codebase', additional_kwargs={}, response_metadata={}, id='ec40e76d-c074-4330-8ff6-344eb1b1ed85'), AIMessage(content="I'd be happy to help you refactor your codebase! However, to provide the most effective assistance, I'll need some more specific information about your project:\n\n1. **What programming language(s) are you working with?**\n2. **What type of codebase is it?** (web application, API, library, etc.)\n3. **What specific refactoring goals do you have?** For example:\n   - Improving code readability/maintainability\n   - Optimizing performance\n   - Reducing technical debt\n   - Following specific design patterns\n   - Breaking up large functions/classes\n   - Improving testability\n\n4. **Do you have any particular pain points or areas of concern?**\n5. **Would you like to share some code snippets or file structures** that you'd like me to focus on?\n\nOnce I have these details, I can create a structured refactoring plan and provide 

In [25]:
from langchain.agents import create_agent
from langchain.agents.middleware import ToolRetryMiddleware


agent = create_agent(
    model=model,
    # tools=[search_tool, database_tool],
    middleware=[
        ToolRetryMiddleware(
            max_retries=3,  # Retry up to 3 times
            backoff_factor=2.0,  # Exponential backoff multiplier
            initial_delay=1.0,  # Start with 1 second delay
            max_delay=60.0,  # Cap delays at 60 seconds
            jitter=True,  # Add random jitter to avoid thundering herd
        ),
    ],
)

In [28]:
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolEmulator


agent = create_agent(
    model=model,
    # tools=[get_weather, search_database, send_email],
    middleware=[
        # Emulate all tools by default
        LLMToolEmulator(),

        # Or emulate specific tools
        # LLMToolEmulator(tools=["get_weather", "search_database"]),

        # Or use a custom model for emulation
        # LLMToolEmulator(model="anthropic:claude-3-5-sonnet-latest"),
    ],
)

In [30]:
from langchain.agents.middleware import before_model, after_model, wrap_model_call
from langchain.agents.middleware import AgentState, ModelRequest, ModelResponse, dynamic_prompt
from langchain.messages import AIMessage
from langchain.agents import create_agent
from langgraph.runtime import Runtime
from typing import Any, Callable


# Node-style: logging before model calls
@before_model
def log_before_model(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print(f"About to call model with {len(state['messages'])} messages")
    return None

# Node-style: validation after model calls
@after_model(can_jump_to=["end"])
def validate_output(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    last_message = state["messages"][-1]
    if "BLOCKED" in last_message.content:
        return {
            "messages": [AIMessage("I cannot respond to that request.")],
            "jump_to": "end"
        }
    return None

# Wrap-style: retry logic
@wrap_model_call
def retry_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelResponse:
    for attempt in range(3):
        try:
            return handler(request)
        except Exception as e:
            if attempt == 2:
                raise
            print(f"Retry {attempt + 1}/3 after error: {e}")

# Wrap-style: dynamic prompts
@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:
    user_id = request.runtime.context.get("user_id", "guest")
    return f"You are a helpful assistant for user {user_id}. Be concise and friendly."

# Use decorators in agent
agent = create_agent(
    model=model,
    middleware=[log_before_model, validate_output, retry_model, personalized_prompt],
    tools=[],
)

In [33]:
from langchain.agents.middleware import AgentState, AgentMiddleware
from typing_extensions import NotRequired
from typing import Any

class CustomState(AgentState):
    model_call_count: NotRequired[int]
    user_id: NotRequired[str]

class CallCounterMiddleware(AgentMiddleware[CustomState]):
    state_schema = CustomState

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        # Access custom state properties
        count = state.get("model_call_count", 0)

        if count > 10:
            return {"jump_to": "end"}

        return None

    def after_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        # Update custom state
        return {"model_call_count": state.get("model_call_count", 0) + 1}
    
agent = create_agent(
    model=model,
    middleware=[CallCounterMiddleware()],
    tools=[],
)

# Invoke with custom state
result = agent.invoke({
    "messages": [HumanMessage("Hello")],
    "model_call_count": 0,
    "user_id": "user-123",
})

result

{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='71ea7815-487f-4d4a-a8c0-29d9e0d71266'),
  AIMessage(content='Hello! 👋 How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 5, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 5}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '4e06935a-6ebb-4e4c-805e-144e1d298e76', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--6f3e5f83-806f-41bc-87df-279a5e6d7056-0', usage_metadata={'input_tokens': 5, 'output_tokens': 11, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})],
 'model_call_count': 1,
 'user_id': 'user-123'}